In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.data import Data, DataLoader

class EGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(EGNNLayer, self).__init__(aggr='mean')  # Sử dụng trung bình để tổng hợp
        self.linear_edge = nn.Linear(in_channels, out_channels)
        self.linear_node = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index, edge_attr):
        # Lấy thông tin từ các node lân cận
        # Cần phải chỉnh sửa phần này
        edge_embedding = self.linear_edge(edge_attr)
        node_embedding = self.linear_node(x)

        # Tính toán các thông tin từ các node lân cận
        out = self.propagate(edge_index, x=node_embedding, edge_attr=edge_embedding)
        return out

    def message(self, x_j, edge_attr):
        # Tính toán thông điệp từ các node lân cận
        return x_j + edge_attr  # Có thể thay đổi theo bài toán

    def update(self, aggr_out):
        return aggr_out  # Trả về đầu ra đã tổng hợp

class EGNN(nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_dim, num_classes):
        super(EGNN, self).__init__()
        self.layer1 = EGNNLayer(num_node_features + num_edge_features, hidden_dim)
        self.layer2 = EGNNLayer(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # Thực hiện hai lớp EGNN
        x = self.layer1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.layer2(x, edge_index, edge_attr)

        # Sử dụng global pooling
        x = global_mean_pool(x, batch)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)
# Ví dụ về cách sử dụng EGNN
def main():
    # Tạo dữ liệu mẫu (bạn nên thay thế bằng dữ liệu thực tế của bạn)
    num_node_features = 3  # Số lượng đặc trưng của node
    num_edge_features = 2  # Số lượng đặc trưng của cạnh
    num_classes = 2
    hidden_dim = 16

    # Tạo một đồ thị mẫu
    x = torch.rand((4, num_node_features))  # 4 nodes
    edge_index = torch.tensor([[0, 1, 2, 0],
                                [1, 0, 0, 2]], dtype=torch.long)  # Edges
    edge_attr = torch.rand((3, num_edge_features))  # 3 edges
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

    model = EGNN(num_node_features, num_edge_features, hidden_dim, num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Huấn luyện mô hình
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, torch.tensor([0]))  # Giả sử nhãn là 0 cho ví dụ
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}: Loss {loss.item()}')

In [7]:
1

1

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.data import Data, DataLoader

class EGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(EGNNLayer, self).__init__(aggr='mean')  # Sử dụng trung bình để tổng hợp
        self.linear_edge = nn.Linear(in_channels, out_channels)
        self.linear_node = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index, edge_attr):
        # Lấy thông tin từ các node lân cận
        # Cần phải chỉnh sửa phần này
        edge_embedding = self.linear_edge(edge_attr)
        node_embedding = self.linear_node(x)

        # Tính toán các thông tin từ các node lân cận
        out = self.propagate(edge_index, x=node_embedding, edge_attr=edge_embedding)
        return out

    def message(self, x_j, edge_attr):
        # Tính toán thông điệp từ các node lân cận
        return x_j + edge_attr  # Có thể thay đổi theo bài toán

    def update(self, aggr_out):
        return aggr_out  # Trả về đầu ra đã tổng hợp

class EGNN(nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_dim, num_classes):
        super(EGNN, self).__init__()
        self.layer1 = EGNNLayer(num_node_features + num_edge_features, hidden_dim)
        self.layer2 = EGNNLayer(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        # Thực hiện hai lớp EGNN
        x = self.layer1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.layer2(x, edge_index, edge_attr)

        # Sử dụng global pooling
        x = global_mean_pool(x, batch)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)
# Ví dụ về cách sử dụng EGNN
def main():
    # Tạo dữ liệu mẫu (bạn nên thay thế bằng dữ liệu thực tế của bạn)
    num_node_features = 3  # Số lượng đặc trưng của node
    num_edge_features = 2  # Số lượng đặc trưng của cạnh
    num_classes = 2
    hidden_dim = 16

    # Tạo một đồ thị mẫu
    x = torch.rand((4, num_node_features))  # 4 nodes
    edge_index = torch.tensor([[0, 1, 2, 0],
                                [1, 0, 0, 2]], dtype=torch.long)  # Edges
    edge_attr = torch.rand((3, num_edge_features))  # 3 edges
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

    model = EGNN(num_node_features, num_edge_features, hidden_dim, num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Huấn luyện mô hình
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, torch.tensor([0]))  # Giả sử nhãn là 0 cho ví dụ
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}: Loss {loss.item()}')

In [9]:
main()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x2 and 5x16)

In [2]:
import dgl
import pandas as pd
import networkx as nx
import torch
# Giả sử bạn đã có DataFrame `data`
data = pd.DataFrame({
    'IPV4_SRC_ADDR': ['192.168.1.1', '192.168.1.2'],
    'IPV4_DST_ADDR': ['192.168.1.3', '192.168.1.4'],
    'PROTOCOL': ['TCP', 'UDP'],
    'L7_PROTO': ['HTTP', 'DNS'],
    'IN_BYTES': [100, 200],
    'OUT_BYTES': [150, 250],
    'IN_PKTS': [10, 20],
    'OUT_PKTS': [15, 30],
    'TCP_FLAGS': ['SYN', 'ACK'],
    'FLOW_DURATION_MILLISECONDS': [1000, 2000],
    'label': ['attack', 'normal']
})

# Tạo đồ thị NetworkX từ DataFrame
G = nx.from_pandas_edgelist(
    data,
    source='IPV4_SRC_ADDR',
    target='IPV4_DST_ADDR',
    edge_attr=['PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 
               'IN_PKTS', 'OUT_PKTS', 'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'label'],
    create_using=nx.MultiDiGraph()
)

# Chuyển đổi đồ thị NetworkX sang DGL
dgl_graph = dgl.from_networkx(G)

# Thêm thuộc tính cạnh vào đồ thị DGL
for attr in ['PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 
             'IN_PKTS', 'OUT_PKTS', 'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'label']:
    dgl_graph.edata[attr] = torch.tensor([data[attr].values for _, _, data in G.edges(data=True)])

# Kiểm tra thông tin edata
print("DGLGraph Edges:", dgl_graph.edges())
print("Edge Data Schemes:", dgl_graph.edata)


AttributeError: 'str' object has no attribute 'values'

In [ ]:
m = {
    'IPV4_SRC_ADDR': ['192.168.1.1'] * 10,  # Cùng địa chỉ nguồn
    'IPV4_DST_ADDR': ['192.168.1.2'] * 10,  # Cùng địa chỉ đích
    'PROTOCOL': ['TCP'] * 10,
    'L7_PROTO': ['HTTP'] * 10,
    'IN_BYTES': [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500],
    'OUT_BYTES': [500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400],
    'IN_PKTS': [10, 15, 20, 25, 30, 35, 40, 45, 50, 55],
    'OUT_PKTS': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    'TCP_FLAGS': ['SYN'] * 10,
    'FLOW_DURATION_MILLISECONDS': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'label': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # Label nhị phân
}
m = pd.DataFrame(m)

In [ ]:
Gm = nx.from_pandas_edgelist(
    m,
    source='IPV4_SRC_ADDR',
    target='IPV4_DST_ADDR',
    edge_attr=['PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 
                                'IN_PKTS', 'OUT_PKTS', 'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'label'],
    create_using=nx.MultiDiGraph()
)

In [ ]:
Gm.edges(data=True)

In [ ]:
edges = Gm.get_edge_data('192.168.1.1', '192.168.1.2')

# In ra thuộc tính của từng cạnh
for key, attr in edges.items():
    print(f"Edge key: {key}, Attributes: {attr}")

In [ ]:
dgl_graph = dgl.from_networkx(Gm)
print("DGLGraph Edges:", dgl_graph)